In [386]:
import numpy as np
import sys,os
from ase.io import read,write
from ase.visualize import view
import ase
import cPickle as pck
import pandas as pd
import spglib as spg
from tqdm import tqdm_notebook
import re

In [387]:
sys.path.insert(0,'../make_input/')

In [388]:
from qe_input import makeQEInput
from custom_frame import get_ibrav0_frame
from utils import get_symprec

# import info and structures

In [389]:
fileNames = {}
infoPath = '../info/'
structurePath = '../structures/'
fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [390]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [77]:
sg,it = 79,3
cc = crystals[sg][it]
dd,_,_ = get_ibrav0_frame(cc,sg)


# get filenames

In [391]:
import fnmatch
import os

path = '../test_run/check_ibrav0_sym/'
matches = []
for root, dirnames, filenames in os.walk(path):
    for dirname in fnmatch.filter(dirnames, 'sg_80-*'):
        filenames = os.listdir(os.path.join(root,dirname))
        for filename in fnmatch.filter(filenames, 'qe.out'):
            matches.append(os.path.abspath(os.path.join(root,dirname, filename)))

In [40]:
matches

['/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_4-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_1-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_3-0/qe.out']

In [32]:
os.listdir('../test_run/check_ibrav0_sym/sg_80-f_0-0')

['qe.in', 'qe.err', 'qe.out']

In [392]:
def get_filenames(path,fn_pattern='qe.out',dir_pattern=None):
    import fnmatch
    import os
    matches = []
    if dir_pattern is None:
        # Find the paths of all the filename with fn_pattern within path
        for root, dirnames, filenames in os.walk(path):
            for filename in fnmatch.filter(filenames, fn_pattern):
                matches.append(os.path.join(root, filename))
        return matches
    else:
        # Find the paths with dir_pattern and all the filename with fn_pattern within dir_pattern (no recursion)
        for root, dirnames, filenames in os.walk(path):
            for dirname in fnmatch.filter(dirnames, dir_pattern):
                filenames = os.listdir(os.path.join(root,dirname))
                for filename in fnmatch.filter(filenames, fn_pattern):
                    matches.append(os.path.abspath(os.path.join(root,dirname, filename)))


In [44]:
path = '../test_run/check_ibrav0_sym/'
fns = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)

# List pseudo

In [376]:
path = '../pseudo/SSSP_eff_PBE/'
pattern = '*.UPF'
pattern = '*.upf'

In [377]:
fns = get_filenames(path,fn_pattern='*.UPF',dir_pattern=None)
fns += get_filenames(path,fn_pattern='*.upf',dir_pattern=None)

In [346]:
print len(fns)
print fns[0]

86
../pseudo/SSSP_eff_PBE/Hf.pbe-spn-rrkjus_psl.0.2.UPF


In [319]:
from raw_info import z2symb

In [324]:
import re,os,sys
re.search?

In [373]:
print 'he'.capitalize().replace('_','')
print 'h_'.capitalize().replace('_','')

He
H


In [343]:
dic = {}
# for fn in fns:
fn = fns[1]
for z,symb in z2symb.iteritems():
    name = os.path.basename(fn)
    if symb ==  name[:2].capitalize().replace('_',''):
        dic[z] = name
print dic

{2: 'He_ONCV_PBE-1.0.upf'}


In [347]:
dic = {}
for fn in fns:
    for z,symb in z2symb.iteritems():
        name = os.path.basename(fn)
        if symb ==  name[:2].capitalize():
            dic[z] = name
print dic

{2: 'He_ONCV_PBE-1.0.upf', 3: 'li_pbe_v1.4.uspp.F.UPF', 4: 'Be_ONCV_PBE-1.0.upf', 10: 'Ne.pbe-n-kjpaw_psl.1.0.0.UPF', 11: 'Na_pbe_v1.uspp.F.UPF', 12: 'mg_pbe_v1.4.uspp.F.UPF', 13: 'Al.pbe-n-kjpaw_psl.1.0.0.UPF', 14: 'Si.pbe-n-rrkjus_psl.1.0.0.UPF', 17: 'cl_pbe_v1.4.uspp.F.UPF', 18: 'Ar.pbe-n-rrkjus_psl.1.0.0.UPF', 20: 'Ca_pbe_v1.uspp.F.UPF', 21: 'Sc_pbe_v1.uspp.F.UPF', 22: 'ti_pbe_v1.4.uspp.F.UPF', 24: 'cr_pbe_v1.5.uspp.F.UPF', 25: 'Mn.pbe-spn-kjpaw_psl.0.3.1.UPF', 26: 'Fe.pbe-spn-kjpaw_psl.0.2.1.UPF', 27: 'Co_pbe_v1.2.uspp.F.UPF', 28: 'ni_pbe_v1.4.uspp.F.UPF', 29: 'Cu_pbe_v1.2.uspp.F.UPF', 30: 'Zn_pbe_v1.uspp.F.UPF', 31: 'Ga.pbe-dn-rrkjus_psl.0.2.UPF', 32: 'Ge.pbe-dn-kjpaw_psl.1.0.0.UPF', 33: 'As.pbe-n-rrkjus_psl.0.2.UPF', 34: 'Se_pbe_v1.uspp.F.UPF', 35: 'br_pbe_v1.4.uspp.F.UPF', 36: 'Kr.pbe-n-rrkjus_psl.0.2.3.UPF', 37: 'Rb_ONCV_PBE-1.0.upf', 38: 'Sr.pbe-spn-rrkjus_psl.1.0.0.UPF', 40: 'Zr_pbe_v1.uspp.F.UPF', 41: 'Nb.pbe-spn-kjpaw_psl.0.3.0.UPF', 42: 'Mo_ONCV_PBE-1.0.upf', 43: 'Tc_ONCV

In [381]:
def get_pseudo(pseudoPath):
    fns = get_filenames(pseudoPath,fn_pattern='*.UPF',dir_pattern=None)
    fns += get_filenames(pseudoPath,fn_pattern='*.upf',dir_pattern=None)
    
    dic = {}
    for fn in fns:
        for z,symb in z2symb.iteritems():
            name = os.path.basename(fn)
            if symb ==  name[:2].capitalize().replace('_','').replace('.',''):
                dic[z] = name
    return dic
pseudoPath = '../pseudo/SSSP_acc_PBE/'
pp = get_pseudo(pseudoPath)
print pp

{1: 'H.pbe-rrkjus_psl.0.1.UPF', 2: 'He_ONCV_PBE-1.0.upf', 3: 'li_pbe_v1.4.uspp.F.UPF', 4: 'Be_ONCV_PBE-1.0.upf', 5: 'B.pbe-n-kjpaw_psl.0.1.UPF', 6: 'C_pbe_v1.2.uspp.F.UPF', 7: 'N.pbe.theos.UPF', 8: 'O.pbe-n-kjpaw_psl.0.1.UPF', 9: 'f_pbe_v1.4.uspp.F.UPF', 10: 'Ne.pbe-n-kjpaw_psl.1.0.0.UPF', 11: 'Na_pbe_v1.uspp.F.UPF', 12: 'mg_pbe_v1.4.uspp.F.UPF', 13: 'Al.pbe-n-kjpaw_psl.1.0.0.UPF', 14: 'Si.pbe-n-rrkjus_psl.1.0.0.UPF', 15: 'P.pbe-n-rrkjus_psl.1.0.0.UPF', 16: 'S_pbe_v1.2.uspp.F.UPF', 17: 'Cl.pbe-n-rrkjus_psl.1.0.0.UPF', 18: 'Ar.pbe-n-rrkjus_psl.1.0.0.UPF', 19: 'K.pbe-spn-rrkjus_psl.1.0.0.UPF', 20: 'Ca_pbe_v1.uspp.F.UPF', 21: 'Sc_pbe_v1.uspp.F.UPF', 22: 'ti_pbe_v1.4.uspp.F.UPF', 23: 'V_pbe_v1.uspp.F.UPF', 24: 'cr_pbe_v1.5.uspp.F.UPF', 25: 'Mn.pbe-spn-kjpaw_psl.0.3.1.UPF', 26: 'Fe.pbe-spn-kjpaw_psl.0.2.1.UPF', 27: 'Co_pbe_v1.2.uspp.F.UPF', 28: 'ni_pbe_v1.4.uspp.F.UPF', 29: 'Cu_pbe_v1.2.uspp.F.UPF', 30: 'Zn_pbe_v1.uspp.F.UPF', 31: 'Ga.pbe-dn-kjpaw_psl.1.0.0.UPF', 32: 'Ge.pbe-dn-kjpaw_psl.1.

## get wf and rho cutoff from table

In [349]:
pseudoPath = '../pseudo/SSSP_acc_PBE/'
fn_table = pseudoPath + 'cutoff_table.dat'
import pandas as pd

In [359]:
pd.read_csv?

In [363]:
tt = pd.read_csv(fn_table,sep=' ',header=None,names=['symb','wfccutoff','rho_mult','PPtype'],na_values='??')
tt.tail(5)

,symb,wfccutoff,rho_mult,PPtype
80,Ho,40.0,8,Wentzcovitch
81,Er,40.0,8,Wentzcovitch
82,Tm,40.0,8,Wentzcovitch
83,Yb,40.0,8,Wentzcovitch
84,Lu,45.0,8,Wentzcovitch


In [357]:
a,b = np.array(tt[tt['symb']=='He'].values[0][1:3],dtype=np.int32)
print a,b

55 4


In [365]:
wfccutoff,rhocutoff = {},{}
for z,symb in z2symb.iteritems():
    wf_cutoff,mult = np.array(tt[tt['symb']==symb].values[0][1:3],dtype=float)
    wfccutoff[z] = wf_cutoff if wf_cutoff>=50 else 50
    rhocutoff[z] = mult*wfccutoff[z]
print wfccutoff
print rhocutoff

{1: 55.0, 2: 55.0, 3: 50, 4: 55.0, 5: 50, 6: 50.0, 7: 55.0, 8: 70.0, 9: 50.0, 10: 200.0, 11: 50, 12: 50, 13: 50, 14: 50, 15: 50, 16: 50, 17: 60.0, 18: 120.0, 19: 50.0, 20: 50, 21: 50, 22: 50, 23: 50, 24: 50, 25: 70.0, 26: 90.0, 27: 55.0, 28: 50, 29: 50, 30: 50, 31: 70.0, 32: 50, 33: 50, 34: 50, 35: 50, 36: 100.0, 37: 50.0, 38: 50, 39: 50, 40: 50, 41: 50, 42: 50, 43: 50, 44: 50, 45: 50, 46: 55.0, 47: 50, 48: 50, 49: 50, 50: 50, 51: 50, 52: 50, 53: 50, 54: 120.0, 55: 50, 56: 50, 57: 55.0, 58: 50, 59: 50.0, 60: 50, 62: 50, 63: 55.0, 64: 50, 65: 50, 66: 50, 67: 50, 68: 50, 69: 50, 70: 50, 71: 50, 72: 120.0, 73: 50, 74: 50, 75: 50, 76: 60.0, 77: 50, 78: 55.0, 79: 50, 80: 50, 81: 50, 82: 50, 83: 50}
{1: 440.0, 2: 220.0, 3: 400.0, 4: 220.0, 5: 400.0, 6: 400.0, 7: 440.0, 8: 560.0, 9: 400.0, 10: 1600.0, 11: 400.0, 12: 400.0, 13: 400.0, 14: 400.0, 15: 400.0, 16: 400.0, 17: 480.0, 18: 960.0, 19: 400.0, 20: 400.0, 21: 400.0, 22: 400.0, 23: 400.0, 24: 400.0, 25: 840.0, 26: 1080.0, 27: 440.0, 28: 40

# parse files

## some test

In [393]:
import re
import sys
import glob

str_patterns = ['Sym. Ops.', '!','tau']
str_patterns = ['Sym. Ops.']
matches = {key:[] for key in str_patterns}

for fn in fns:
    for line in open(fn, 'r'):
        for str_pattern in str_patterns:
            if re.search(str_pattern, line):
                matches[str_pattern].append(line.replace('\n',''))
print matches['Sym. Ops.']

[]


In [394]:
def get_patterns(fn,str_patterns):
    matches = {key:[] for key in str_patterns}
    
    for line in open(fn, 'r'):
        for str_pattern in str_patterns:
            if re.search(str_pattern, line):
                matches[str_pattern].append(line.replace('\n',''))
    return matches

In [64]:
cc = read(fns[0])
view(cc)

In [129]:
fn = fns[0]
ms = re.search("sg_(.*)-f_",fn)
sg = int(ms.group(1))
print sg

79


In [409]:
def fn2num(fn,pattern="sg_(.*)-f_"):
    ms = re.search(pattern,fn)
    num = int(ms.group(1))
    return num
# fn2sg(fn)

In [83]:
re.findall("[-\d]+", "he33llo 42 I\'m a 32.3 string -30 1 -2")

['33', '42', '32', '3', '-30', '1', '-2']

In [84]:
s = """4.5 abc -4.5 abc - 4.5 abc + .1e10 abc . abc 1.01e-2 abc 
       1.01e-.2 abc 123 abc .123"""
print re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", s)


['4.5', '-4.5', '- 4.5', '+ .1e10', ' 1.01e-2', '       1.01', '-.2', ' 123', ' .123']


[4.5, -4.5, 4.5, 1000000000.0, 0.0101, 123.0, 0.123]


In [396]:
def extract_floats(s):
    l = []
    for t in s.split():
        try:
            l.append(float(t))
        except ValueError:
            pass
    return l

In [89]:
%%timeit -r 3 -n 20
extract_float(s)

20 loops, best of 3: 16.3 µs per loop


## get energy

In [14]:
path = '/scratch/musil/qmat/data/check_ktp_conv/'
fns = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)

In [39]:
params = {'sg':[],'f':[],'Nkpt':[]}
ppp = []
pppp = []

for fn in fns:
    sg = fn2num(fn,pattern="sg_(.*)-f_")
    f = fn2num(fn,pattern="-f_(.*)-Nkpt_")
    Nkpt = fn2num(fn,pattern="-Nkpt_(.*)-")
    ppp.append([sg,f,Nkpt])
    if Nkpt == 1000:
        pppp.append([sg,f])
    params['sg'].append(sg)
    params['f'].append(f)
    params['Nkpt'].append(Nkpt)
    
# print params

In [31]:
Ryd2eV = 13.605698066

In [397]:
def finishedProperly(fn):
    str_patterns = ['JOB DONE.']
    s = get_patterns(fn,str_patterns)['JOB DONE.']
    if s:
        return True
    else:
        return False

In [46]:
str_patterns = ['total energy','number of atoms/cell']
energies = {}
for fn,(sg,f,Nkpt) in zip(fns,ppp):
    if finishedProperly(fn):
        
        s = get_patterns(fn,str_patterns)
        nat = extract_floats(s['number of atoms/cell'][0])[0]
        en = extract_floats(s['total energy'][0])[0]*Ryd2eV / nat
        energies[(sg,f,Nkpt)] = en 
        

In [423]:
def fn2info(fn):
    dirnames = fn.split('/')
    for dirname in dirnames:
        if re.search('sg_',dirname) and re.search('f_',dirname):
            dic = {}
            for info in dirname.split('-'):
                if re.search('_',info):
                    name,num = info.split('_')
                    num = int(num)
                    dic[name] = num
    return dic

ff = '../asdfs/sdf/sg_5-f_3-Nkpt_20-0/qe.out'             
fn2info(ff)

{'Nkpt': 20, 'f': 3, 'sg': 5}

In [416]:
fn = 'sg_5-f_3-Nkpt_20-0'
dic = {}
afs = fn.split('-')
for af in afs:
    if re.search('_',af):
        name,num = af.split('_')
        num = int(num)
        dic[name] = num
print dic

{'Nkpt': 20, 'sg': 5, 'f': 3}


In [ ]:
def get_energy_per_atom():

In [47]:
diff = {}
for sg,f in pppp:
    try:
        diff[(sg,f)] = abs(energies[(sg,f,1000)]-energies[(sg,f,2000)])*100
        print diff[(sg,f)]
    except:
        print '####'
        print sg,f
        print '####'

0.895396091857
0.120406601283
0.174191484723
####
4 0
####
####
227 0
####
3.07015127928
23.9778115068
0.947643673146
####
144 4
####
0.682250926673
0.280301190134
2.40135922251
0.956016279594
####
7 4
####
2.18386080085
0.208126363316
3.29193719654
####
4 4
####
0.00991175103877
0.320363168086
0.603300462217
0.701601630746
####
7 3
####
2.37418637586
1.32515757596
1.28310746537
####
227 1
####
2.33067875487
3.80035718949
1.19834566713
####
7 2
####
0.168243527051
0.183382133767
0.435205464038
####
144 2
####
4.07706987676
0.677220219811
2.15241236358
1.10605481572
####
4 1
####
0.941638458161
2.20734423571
####
144 3
####
2.71710042159
####
144 0
####
####
7 1
####
####
69 1
####
0.437355164331
0.412030424999
0.125442835454
3.10982719555
0.495755922569
2.49346186176
3.80718044707
0.927359278043
0.829297909945
2.59132594724
0.967657655002
0.276666768036
0.343443534146
2.84096329536
####
229 1
####
0.478976695427
0.576519346288
0.715050863286
0.608375387597
0.213441089122
0.101219590761

In [59]:
failed = {}
for fn,(sg,f,Nkpt) in zip(fns,ppp):
    if not finishedProperly(fn):
        with open(fn,'r') as fout:
            failed[(sg,f,Nkpt,fn)] = fout.read()
print len(failed)

45


In [60]:
aa = failed.items()

In [80]:
it = 18
for it in range(len(failed)):
    if aa[it][0][0] not in [144,7,227,4]:
        print it, aa[it][0]
        #print aa[it][1]

1 (69, 3, 1000, '/scratch/musil/qmat/data/check_ktp_conv/sg_69-f_3-Nkpt_1000-0/qe.out')
11 (229, 2, 2000, '/scratch/musil/qmat/data/check_ktp_conv/sg_229-f_2-Nkpt_2000-0/qe.out')
16 (229, 0, 2000, '/scratch/musil/qmat/data/check_ktp_conv/sg_229-f_0-Nkpt_2000-0/qe.out')
19 (229, 1, 1000, '/scratch/musil/qmat/data/check_ktp_conv/sg_229-f_1-Nkpt_1000-0/qe.out')
22 (69, 3, 2000, '/scratch/musil/qmat/data/check_ktp_conv/sg_69-f_3-Nkpt_2000-0/qe.out')
30 (229, 2, 1000, '/scratch/musil/qmat/data/check_ktp_conv/sg_229-f_2-Nkpt_1000-0/qe.out')
41 (69, 1, 2000, '/scratch/musil/qmat/data/check_ktp_conv/sg_69-f_1-Nkpt_2000-0/qe.out')
43 (229, 1, 2000, '/scratch/musil/qmat/data/check_ktp_conv/sg_229-f_1-Nkpt_2000-0/qe.out')
44 (229, 3, 2000, '/scratch/musil/qmat/data/check_ktp_conv/sg_229-f_3-Nkpt_2000-0/qe.out')


## get number of detected symmetries

In [200]:
path = '../test_run/check_ibrav0_sym/'
fns_out = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)
fns_in = get_filenames(path,fn_pattern='qe.in',dir_pattern=None)

In [201]:
fn = fns_in[14]
print fn

../test_run/check_ibrav0_sym/sg_70-f_4-0/qe.in


In [202]:
cc = read(fn)
sg = fn2sg(fn)
symprec = get_symprec(cc,sg)
syms = spg.get_symmetry(cc,symprec=symprec)
Nsym_s = syms['rotations'].shape[0]
print Nsym_s

8


In [164]:
view(cc)

In [154]:
from raw_info import missClassificationCorrection
missClassificationCorrection[70]

KeyError: 70

In [197]:
sg,it = 43,1
cc = crystals[sg][it]
dd,_,_ = get_ibrav0_frame(cc,sg)
view(dd)
print dd.get_cell()

[[ 0.          7.186617    1.52812703]
 [ 1.62630392  0.          1.52812703]
 [ 1.62630392  7.186617    0.        ]]


In [198]:
fn

'../test_run/check_ibrav0_sym/sg_43-f_1-0/qe.out'

In [203]:

# print get_patterns(fn,str_patterns=['Sym. Ops.']).values()
# s = get_patterns(fn,str_patterns=['Sym. Ops.']).values()[0][0]
# Nsym_f = extract_floats(s)[0]
fn = '../test_run/check_ibrav0_sym/sg_43-f_1-0/qe.in'
cc = read(fn)
sg = fn2sg(fn)
# view(cc)
print cc.get_cell()
print sg
symprec = get_symprec(cc,sg)
print symprec
# syms = spg.get_symmetry(cc,symprec=symprec)

# Nsym_s = syms['rotations'].shape[0]

# print Nsym_s == Nsym_f
# print s


[[ 0.          7.186617    1.52812703]
 [ 1.62630392  0.          1.52812703]
 [ 1.62630392  7.186617    0.        ]]
43
1e-05


In [236]:
fn = fns_out[5]
print fn
ss = get_patterns(fn,str_patterns=['Sym. Ops.','warning: symmetry operation #','No symmetry found',
                                          'additional sym.ops. were found but ignored'])
Nsym_f = 0
print ss['additional sym.ops. were found but ignored']
if ss['additional sym.ops. were found but ignored']:
    s = ss['additional sym.ops. were found but ignored'][0]
    Nsym_f += extract_floats(s)[0]
print Nsym_f
# if ss['warning: symmetry operation #']:
#     nums = []
#     for s in ss['warning: symmetry operation #']:
#         nums.append(extract_floats(s)[0])
    
#     nums = np.array(nums).max()
#     Nsym_f = nums.max()
# else:
#     print ss['Sym. Ops.']
#     s = ss['Sym. Ops.'][0][0]
#     Nsym_f = extract_floats(s)[0]

# cc = read(fn)
# sg = fn2sg(fn)
# symprec = get_symprec(cc,sg)
# syms = spg.get_symmetry(cc,symprec=symprec)
# Nsym_s = syms['rotations'].shape[0]

# print Nsym_s == Nsym_f
# print s


../test_run/check_ibrav0_sym/sg_40-f_0-0/qe.out
['          (note:  4 additional sym.ops. were found but ignored']
4.0


In [273]:
path = '../test_run/check_ibrav0_sym/'
fns_out = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)
fns_in = get_filenames(path,fn_pattern='qe.in',dir_pattern=None)

good = []
to_check = []
pbar = tqdm_notebook(total=len(fns_out))
for it,(fn_in,fn_out) in enumerate(zip(fns_in,fns_out)):

    ss = get_patterns(fn_out,str_patterns=['Sym. Ops.','warning: symmetry operation #','No symmetry found',
                                          'additional sym.ops. were found but ignored'])
    Nsym_f = 0
    if ss['additional sym.ops. were found but ignored']:
        s = ss['additional sym.ops. were found but ignored'][0]
        Nsym_f += extract_floats(s)[0]
    if ss['warning: symmetry operation #']:
        nums = []
        for s in ss['warning: symmetry operation #']:
            nums.append(extract_floats(s)[0])

        nums = np.array(nums).max()
        Nsym_f = nums.max()
        
    elif ss['No symmetry found']:
        Nsym_f += 1
    else:

        s = ss['Sym. Ops.'][0]
        
        Nsym_f += extract_floats(s)[0]

    cc = read(fn_in)
    sg = fn2sg(fn_in)
    
    symprec = get_symprec(cc,sg)
    syms = spg.get_symmetry(cc,symprec=symprec)
    Nsym_s = syms['rotations'].shape[0]

    if Nsym_s == Nsym_f:
        good.append(fn_out)
    else:
        to_check.append([it,fn_out,sg,Nsym_s,Nsym_f])

    pbar.update()
pbar.close()

In [274]:
len(to_check)

49

In [277]:
to_check

[[8, '../test_run/check_ibrav0_sym/sg_73-f_8-0/qe.out', 73, 8, 4.0],
 [16, '../test_run/check_ibrav0_sym/sg_211-f_3-0/qe.out', 211, 24, 48.0],
 [17, '../test_run/check_ibrav0_sym/sg_204-f_1-0/qe.out', 204, 24, 48.0],
 [19, '../test_run/check_ibrav0_sym/sg_36-f_7-0/qe.out', 36, 8, 4.0],
 [26, '../test_run/check_ibrav0_sym/sg_15-f_8-0/qe.out', 15, 4, 2.0],
 [27, '../test_run/check_ibrav0_sym/sg_64-f_3-0/qe.out', 64, 8, 4.0],
 [30, '../test_run/check_ibrav0_sym/sg_121-f_4-0/qe.out', 121, 8, 16.0],
 [34, '../test_run/check_ibrav0_sym/sg_204-f_0-0/qe.out', 204, 24, 48.0],
 [38, '../test_run/check_ibrav0_sym/sg_64-f_1-0/qe.out', 64, 8, 4.0],
 [43, '../test_run/check_ibrav0_sym/sg_88-f_1-0/qe.out', 88, 8, 1],
 [45, '../test_run/check_ibrav0_sym/sg_74-f_2-0/qe.out', 74, 8, 4.0],
 [50, '../test_run/check_ibrav0_sym/sg_68-f_5-0/qe.out', 68, 8, 4.0],
 [57, '../test_run/check_ibrav0_sym/sg_119-f_0-0/qe.out', 119, 8, 16.0],
 [67, '../test_run/check_ibrav0_sym/sg_15-f_5-0/qe.out', 15, 4, 2.0],
 [83,

In [275]:
print to_check[4]

[26, '../test_run/check_ibrav0_sym/sg_15-f_8-0/qe.out', 15, 4, 2.0]


In [282]:
it = 43
fn = fns_out[it]
print fn
ss = get_patterns(fn,str_patterns=['Sym. Ops.','warning: symmetry operation #','No symmetry found'])

if ss['warning: symmetry operation #']:
    nums = []
    for s in ss['warning: symmetry operation #']:
        nums.append(extract_floats(s)[0])
    
    nums = np.array(nums).max()
    Nsym_f = nums.max()
elif ss['No symmetry found']:
    Nsym_f += 1
else:
#     print ss['Sym. Ops.']
    s = ss['Sym. Ops.'][0]
    Nsym_f = extract_floats(s)[0]
    
fn = fns_in[it]
cc = read(fn)
sg = fn2sg(fn)
if sg == 36:
    Nsym_s = 4
else:
    symprec = get_symprec(cc,sg)
    syms = spg.get_symmetry(cc,symprec=symprec)
    Nsym_s = syms['rotations'].shape[0]

print Nsym_s == Nsym_f
print s
print Nsym_s

../test_run/check_ibrav0_sym/sg_88-f_1-0/qe.out
False
      8 Sym. Ops. (no inversion) found ( 4 have fractional translation)
8


In [268]:
view(cc)

In [254]:
sg,it = 211,3
cc = crystals[sg][it]
dd,_,_ = get_ibrav0_frame(cc,sg)
view(dd)
view(cc)
print dd.get_cell()

[[-5.52491889  5.52491889  5.52491889]
 [ 5.52491889 -5.52491889  5.52491889]
 [ 5.52491889  5.52491889 -5.52491889]]


In [279]:
missClassificationCorrection[121]

KeyError: 121

In [256]:

symprec = get_symprec(dd,sg)
syms = spg.get_symmetry(dd,symprec=symprec)
Nsym_s = syms['rotations'].shape[0]
print Nsym_s

24


In [261]:
it = 16
fn = fns_in[it]
cc = read(fn)
sg = fn2sg(fn)
symprec = get_symprec(cc,sg)
syms = spg.get_symmetry(cc,symprec=symprec)
Nsym_s = syms['rotations'].shape[0]
print Nsym_s

24


In [262]:
view(cc)